# प्राणी तज्ञ प्रणालीची अंमलबजावणी

[AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners) मधील एक उदाहरण.

या नमुन्यात, आपण काही भौतिक वैशिष्ट्यांवर आधारित प्राणी ठरवण्यासाठी एक सोपी ज्ञान-आधारित प्रणाली अंमलात आणू. प्रणाली खालील AND-OR वृक्षमुळे दर्शविली जाऊ शकते (हे संपूर्ण वृक्षकडील एक भाग आहे, आपण सहजपणे आणखी काही नियम जोडू शकतो):

![](../../../../../../translated_images/mr/AND-OR-Tree.5592d2c70187f283.webp)


## मागे जाणाऱ्या अनुमानांसह आपल्या स्वत: च्या तज्ञ प्रणाली शेल

चल, उत्पादन नियमांवर आधारित ज्ञान प्रतिनिधीकरणासाठी एक सोपी भाषा परिभाषित करण्याचा प्रयत्न करूया. नियम परिभाषित करण्यासाठी आम्ही Python वर्गांना कीवर्ड म्हणून वापरणार आहोत. मूलतः 3 प्रकारचे वर्ग असतील:
* `Ask` वापरकर्त्याला विचारायचा प्रश्न दर्शवतो. त्यात शक्य उत्तरे संच असतो.
* `If` नियम दर्शवतो, आणि तो केवळ नियमाच्या सामग्रीस संचयित करण्यासाठी एक वाक्यरचनात्मक साखर आहे
* `AND`/`OR` ही वृक्षाच्या AND/OR शाखांचे प्रतिनिधीत्व करणारे वर्ग आहेत. ते फक्त आतल्या तर्कांची यादी संग्रहित करतात. कोड सोपे करण्यासाठी, सर्व कार्यक्षमता पालक वर्ग `Content` मध्ये परिभाषित केली आहे


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

आपल्या प्रणालीमध्ये, कार्यशील स्मृतीमध्ये **घटनेची** यादी **गुणधर्म-मूल्य जोड्यां** स्वरूपात असेल. ज्ञानकोशाला एका मोठ्या शब्दकोशाप्रमाणे परिभाषित करता येईल जो क्रियांना (नवीन घटक जे कार्यशील स्मृतीमध्ये समाविष्ट केले जाणे आवश्यक आहे) अटींशी जोडतो, जे AND-OR अभिव्यक्तींमध्ये व्यक्त केल्या जातात. तसेच, काही घटक `Ask` केले जाऊ शकतात.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

बॅकवॉर्ड इन्फरन्स करण्यासाठी, आपण `Knowledgebase` क्लास परिभाषित करू. यामध्ये असेल:
* कार्यरत `memory` - एक डिक्शनरी जी अॅट्रिब्युट्सना मूल्ये मॅप करते
* Knowledgebase मधील `rules` वरिभाषित स्वरूपात

मुख्य दोन मेथड्स आहेत:
* `get` जे एका अॅट्रिब्युटचे मूल्य प्राप्त करते, गरज भासल्यास इन्फरन्स करतो. उदाहरणार्थ, `get('color')` रंगाच्या स्लॉटचे मूल्य मिळवेल (गरज भासल्यास विचारेल, आणि नंतरच्या वापरासाठी कार्यरत मेमरीमध्ये मूल्य साठवेल). जर आपण `get('color:blue')` विचारले, तर रंगासाठी विचारेल, आणि नंतर रंगानुसार `y`/`n` मूल्य परत करेल.
* `eval` प्रत्यक्ष इन्फरन्स करतो, म्हणजे AND/OR ट्रीमध्ये फिरतो, उप-लक्ष्यांचे मूल्यांकन करतो, इत्यादी.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

आता आपण आपला प्राणी ज्ञानसंच परिभाषित करूया आणि सल्ला प्रक्रिया पार पाडूया. लक्षात ठेवा की या कॉलमध्ये तुम्हाला प्रश्न विचारले जातील. तुम्ही हो वा नाही या प्रश्नांसाठी `y`/`n` टाइप करून उत्तर देऊ शकता, किंवा जास्त पर्याय असलेल्या प्रश्नांसाठी 0..N या क्रमांकावर उत्तर देऊ शकता.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## पुढील अनुमानासाठी Experta वापरणे

पुढील उदाहरणात, आपण ज्ञान सादरीकरणासाठी असलेल्या लायब्ररींपैकी एका, [Experta](https://github.com/nilp0inter/experta) वापरून पुढील अनुमान अमलात आणण्याचा प्रयत्न करू. **Experta** ही पायथनमधील पुढील अनुमान प्रणाली तयार करण्यासाठीची लायब्ररी आहे, जी पारंपरिक जुन्या प्रणाली [CLIPS](http://www.clipsrules.net/index.html) प्रमाणे डिझाइन केलेली आहे.

आपणही अनेक अडचणीशिवाय पुढील चेनिंग स्वतः अमलात आणू शकतो होतो, पण साध्या अमलबजावणी सहसा फार कार्यक्षम नसतात. अधिक प्रभावी नियम जुळवणीसाठी एक विशेष अल्गोरिदम [Rete](https://en.wikipedia.org/wiki/Rete_algorithm) वापरला जातो.


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/nilp0inter/experta

  Cloning https://github.com/nilp0inter/experta to /tmp/pip-req-build-7qurtwk3
  Running command git clone --filter=blob:none --quiet https://github.com/nilp0inter/experta /tmp/pip-req-build-7qurtwk3
  Resolved https://github.com/nilp0inter/experta to commit c6d5834b123861f5ae09e7d07027dc98bec58741
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for experta: filename=experta-1.9.5.dev1-py3-none-any.whl size=34804 sha256=888c459512a5e713f4b674caa9a0f96cfdf07ec0d6eb56cc318ce0653d218014
  Stored in directory: /tmp/pip-ephem-wheel-cache-1eeii9zy/wheels/3d/e8/bb/22d7956359603fa8dd679aa09f5b8efb3f29991c3986fdc787
Successfully built experta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [experta]


In [13]:
from experta import *
#import experta

आपण आपल्या सिस्टीमला `KnowledgeEngine` ची उपवर्ग वर्ग म्हणून परिभाषित करू. प्रत्येक नियम स्वतंत्र फंक्शनने परिभाषित केला जातो ज्यावर `@Rule` अ‍ॅनोटेशन असते, जे दर्शवते की नियम कधी लागू होईल. नियमाच्या आत, आपण `declare` फंक्शन वापरून नवीन तथ्ये जोडू शकतो, आणि त्या तथ्यांमुळे पुढील नियम फॉरवर्ड इन्फरन्स इंजिनद्वारे कॉल केले जातील.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

एकदा आपण ज्ञानभांडार परिभाषित केल्यानंतर, आपण आपली कार्यरत स्मृती काही प्रारंभिक तथ्यांनी भरतो, आणि नंतर `run()` मेथड कॉल करतो ज्यामुळे तपासणी केली जाते. तुम्हाला परिणाम म्हणून दिसेल की नवीन निष्पन्न तथ्ये कार्यरत स्मृतीमध्ये जोडली जातात, ज्यात प्राण्याविषयी अंतिम तथ्यही समाविष्ट आहे (जर आपण सर्व प्रारंभिक तथ्ये योग्यरित्या सेट केली असतील तर).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**सूचना**:
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) वापरून भाषांतरित केला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी, कृपया लक्षात ठेवा की स्वयंचलित भाषांतरांमध्ये चुका किंवा अचूकता नसण्याची शक्यता असते. मूळ दस्तऐवज त्याच्या मूळ भाषेत अधिकृत स्रोत मानला जावा. महत्त्वपूर्ण माहितीसाठी, व्यावसायिक मानवी भाषांतर करण्याचा सल्ला दिला जातो. या भाषांतराच्या वापरामुळे उद्भवणाऱ्या कोणत्याही गैरसमजुतीसाठी किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार नाही.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
